In [50]:
# Primero leo los CSV para poder armar los datasets
import pandas as pd

df_train = pd.read_csv("./fashion-mnist_train.csv")
df_test = pd.read_csv("./fashion-mnist_test.csv")

In [51]:
# Construyo mis datasets. En un principio tomo únicamente el csv de train provisto por Kaggle
# pero la idea es hacer k-fold cross-validation con lo cual luego voy a estar variando esto
x_train = df_train.drop(columns=["label"]).values
y_train = df_train["label"].values

x_test = df_test.drop(columns=["label"]).values
y_test = df_test["label"].values

In [52]:
# Transformo mi dataset con PCA para reducir la dimensión manteniendo la información más importante
from sklearn.decomposition import PCA
pca = PCA(n_components=50) # n_components es la cantidad de componentes que mantiene como mínimo la transformación. 
# Arranqué con una matriz de 60000x784 así que llegar a 60000x50 está bastante bien -> la idea es ir variando esto
# a ver cómo va variando el resultado
x_train_transformed = pca.fit_transform(x_train)
x_test_transformed = pca.fit_transform(x_test)

In [ ]:
# Habiendo hecho la tranformación, armo el modelo
from sklearn.neighbors import KNeighborsClassifier
# n_neighbors es el número de vecinos entre los cuales debe haber 
# una mayoría para setear el label correspondiente 
# (ej. si 3 de los 5 vecinos tienen label X entonces el nuevo punto es X)
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train_transformed, y_train)
knn.predict(x_test_transformed)
knn.score(x_test_transformed, y_test)